In [3]:
import os, sys
sys.path.append("C:/Users/piyalong/CrowdCounting-P2PNet/")
sys.path.append('C:/Users/piyalong/multi-object-tracker/')

import argparse
import datetime
import random
import time
from pathlib import Path

import torch
import torchvision.transforms as standard_transforms
import numpy as np

from PIL import Image
import cv2
from crowd_datasets import build_dataset
from engine import *
from models import build_model

import warnings
warnings.filterwarnings('ignore')

from matplotlib import pyplot as plt

torch.cuda.empty_cache() 


In [4]:
class Args():
    backbone='vgg16_bn'
    gpu_id=0
    img_path='vis/test.jpg'
    line=2
    output_dir='output'
    row=2
    weight_path='C:/Users/piyalong/CrowdCounting-P2PNet//weights/SHTechA.pth'

args=Args()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch. set_grad_enabled(False)
# def main(args, debug=False):

#     os.environ["CUDA_VISIBLE_DEVICES"] = '{}'.format(args.gpu_id)

print(args)
# device = torch.device('cpu')
# get the P2PNet
model = build_model(args)
# move to GPU
model.to(device)
# load trained model
if args.weight_path is not None:
    checkpoint = torch.load(args.weight_path, map_location=device)
    model.load_state_dict(checkpoint['model'])
# convert to eval mode
model.eval()
# create the pre-processing transform
transform = standard_transforms.Compose([
    standard_transforms.ToTensor(), 
    standard_transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [5]:
person_width = 20
person_height = 40

def point_to_box(points,scores):
    ps=[]
    for p in points:
        top,left,bottom,right = p[1]-person_height,p[0]-person_width,p[1]+person_height,p[0]+person_width
        ps.append( [int(left),int(top),person_width*2,person_height*2])
    return(ps)
# point_to_box(points,scores)

In [6]:
from motrackers import CentroidTracker, CentroidKF_Tracker, SORT, IOUTracker
from motrackers.utils import draw_tracks
def get_trace(file_path):
    # file_path='C:/Users/piyalong/Traffic Management/2021season/Passageway(Mobile)_2021-10-02_21h20min02s757ms.mp4'
    # file_path='C:/Users/piyalong/Traffic Management/2021season/maroonwhite/WCG _ PTZ PW Inside West (SC2)-2021-04-24_15h00min00s000ms.mp4'
#     file_path='D:/Appa State/WCG _ PTZ PW Outside East View (SC4)-2022-09-10_18h00min00s000ms.mp4'

    video_capture = cv2.VideoCapture(file_path)
    w = int(video_capture.get(3))
    h = int(video_capture.get(4))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))
    new_width = w // 128 * 128
    new_height = h // 128 * 128
    out = cv2.VideoWriter(file_path[:-4]+'_CentroidKF_Tracker.avi', fourcc, fps, (new_width, new_height))

    tracker = CentroidKF_Tracker(max_lost=10,time_step=10)

    # tracker = CentroidTracker(max_lost=fps)

    data={}
    frame_number=0
    while True:
        ret, frame = video_capture.read()  # frame shape 640*480*3
        if ret != True:
            break
        img_raw = frame
        img_raw = cv2.cvtColor(img_raw, cv2.COLOR_BGR2RGB)
        img_raw = Image.fromarray(img_raw)
        # round the size
        width, height = img_raw.size
        new_width = width // 128 * 128
        new_height = height // 128 * 128
        img_raw = img_raw.resize((new_width, new_height), Image.ANTIALIAS)
        # pre-proccessing
        img = transform(img_raw)

        samples = torch.Tensor(img).unsqueeze(0)
        samples = samples.to(device)
        # run inference
        outputs = model(samples)
        outputs_scores = torch.nn.functional.softmax(outputs['pred_logits'], -1)[:, :, 1][0]

        outputs_points = outputs['pred_points'][0]

        threshold = 0.5
        # filter the predictions
        points = outputs_points[outputs_scores > threshold].detach().cpu().numpy().tolist()
        predict_cnt = int((outputs_scores > threshold).sum())
        scores= outputs_scores[outputs_scores > threshold].detach().cpu().numpy().tolist()

        outputs_scores = torch.nn.functional.softmax(outputs['pred_logits'], -1)[:, :, 1][0]

        outputs_points = outputs['pred_points'][0]

        detection_bboxes = point_to_box(points,scores)

        detection_confidences = np.array(scores)

        detection_class_ids =np.ones(len(detection_confidences))
        tracks = tracker.update(np.array(detection_bboxes), detection_confidences, detection_class_ids)
        # output_tracks = tracker.track(detection_bboxes, detection_confidences, detection_class_ids)

        # draw the predictions
        size = 2
        img_to_draw = cv2.cvtColor(np.array(img_raw), cv2.COLOR_RGB2BGR)


        for t in tracks:
            frame_numeber=t[0]
            ID = t[1]
            xmin,ymin,w,h = t[2:6]
            xcentroid, ycentroid = int(xmin + 0.5*w), int(ymin + 0.5*h)
            if ID in data.keys():
                data[ID].append([xcentroid, ycentroid,frame_number])
            else:
                data.update({ID:[]})
                data[ID].append([xcentroid, ycentroid,frame_number])

        img_to_draw=cv2.putText(img_to_draw,"Frame:"+str(frame_number)+' FPS:'+str(fps),(100,50),cv2.FONT_HERSHEY_SIMPLEX,1, (255,0,0), 3)

        updated_image = img_to_draw

        updated_image = draw_tracks(updated_image, tracks)

        updated_image = cv2.putText(updated_image,"Crowd Estimation: "+str(predict_cnt), (100,100), cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 3)

        # cv2.imshow("Output", updated_image)
        out.write(updated_image)    
        frame_number+=1
        if cv2.waitKey(1) & 0xFF == ord('q'):

            break
    out.release()
    cv2.destroyAllWindows()
    np.save(file_path[:-4]+'.npy',data)

In [7]:
import os

In [ ]:
for game in  ['Ole Miss','Florida','LSU']:

    for file in os.listdir('D:/2022 season/'+game+"/"):
        if file.endswith('.mp4') and "Outside East View" in file:
            print(os.path.join('D:/2022 season/',game,file))
            file_path=os.path.join('D:/2022 season/',game,file)
            get_trace(file_path)

D:/2022 season/Ole Miss\WCG _ PTZ PW Outside East View (SC4)-2022-10-29_16h00min00s000ms.mp4
D:/2022 season/Ole Miss\WCG _ PTZ PW Outside East View (SC4)-2022-10-29_17h00min00s000ms.mp4
D:/2022 season/Ole Miss\WCG _ PTZ PW Outside East View (SC4)-2022-10-29_18h00min00s000ms.mp4
D:/2022 season/Ole Miss\WCG _ PTZ PW Outside East View (SC4)-2022-10-29_21h00min00s000ms.mp4
D:/2022 season/Ole Miss\WCG _ PTZ PW Outside East View (SC4)-2022-10-29_22h00min00s000ms.mp4
D:/2022 season/Ole Miss\WCG _ PTZ PW Outside East View (SC4)-2022-10-29_23h00min00s000ms.mp4
D:/2022 season/Florida\WCG _ PTZ PW Outside East View (SC4)-2022-11-05_14h00min00s000ms.mp4
D:/2022 season/Florida\WCG _ PTZ PW Outside East View (SC4)-2022-11-05_15h00min00s000ms.mp4
D:/2022 season/LSU\WCG _ PTZ PW Outside East View (SC4)-2022-11-26_21h00min00s000ms.mp4


In [6]:
get_trace('D:/2021season/Passageway(Mobile)_2021-10-02_21h20min02s757ms.mp4')

In [45]:
fps

14

Python 3.6.15
